# Training of pre-trained Resnet in U-shape for semantic SEGMENTATION


"""
ABOUT
This script is used to train anew version of the IDENTIFICATION model.

INFO
Before start, check the initialization to modify the parameters.
"""

In [1]:
import tensorflow as tf
from keras import models, layers, optimizers
import numpy as np
import os

from src.data.ide_loader \
    import images_labels_loader, input_creator

2022-06-18 17:52:53.196011: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-18 17:52:53.196037: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


0
270336
270336
270336


------------------------------------------
## Parameters

In [2]:
## path with all raw data
path_input = '../data/raw/segmentation/'
## targets: Name of specific image, position in json to be considered
image = (["Potsdam_01"])
## number of augmented versions used for each image/target:
num_aug = 2

# Processing specification:
## in pixel:
tile_size = 25
## in pixel per side, expands small tile to large tile:
border = 15
## max number of neg per pos tiles in training set:
max_neg_per_true = 3

# Model specification:
path_model = '../models/segmentation_options/'
model_name = "test"
path_model = path_model + model_name + '/'
batch_size = 25
epochs = 2

In [3]:
# make the model_name directory
if not os.path.exists(f"{path_model}"):
    os.mkdir(f"{path_model}")

------------------------------------
## Load data

In [4]:
data_path = f"../data/raw/segmentation/Dresden_01/"
file_name_prefix = "task-10-annotation-9-by-1-tag-task-10-annotation-9-by-1-tag-buildings-0.npy"
tags = np.load(f"{data_path}{file_name_prefix}")



FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/segmentation/Dresden_01/task-10-annotation-9-by-1-tag-task-10-annotation-9-by-1-tag-buildings-0.npy'

---------------------------------------------------------
## Make the model

In [21]:
# load pre-build resnet50 unet
path_model = "../models/segmentation_options/512_resnet50_unet.h5"
resnet = tf.keras.models.load_model(path_model)
resnet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [22]:
model = resnet

metrics = [
      tf.keras.metrics.BinaryAccuracy(name='binary_crossentropy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Precision(name='binary_accuracy'),
      tf.keras.metrics.Recall(name='F1Score'),
      ]
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer="Adam",
    metrics=metrics,
    )


# Model weights are saved at the end of every epoch,
    # if it's the best seen so far.
checkpoint_filepath = '../models/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='F1Score',
    mode='max',
    save_best_only=True)

# train model
history = model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_validate,
    callbacks=[model_checkpoint_callback],
)

# The model weights (that are considered the best)
#  are loaded into the model.
model.load_weights(checkpoint_filepath)


# save model
print(f"\nModel is trained and saved as {model_name}."
      f"   You can find the model in {path_model}."
      f"---To use it for future predictions - replace default!---")
model.save(f"{path_model}")

Epoch 1/2


ValueError: in user code:

    File "/home/luise/Documents/DataScience/Projects/UrbanEagle/urbaneagle_visionai/venv/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/luise/Documents/DataScience/Projects/UrbanEagle/urbaneagle_visionai/venv/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/luise/Documents/DataScience/Projects/UrbanEagle/urbaneagle_visionai/venv/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/luise/Documents/DataScience/Projects/UrbanEagle/urbaneagle_visionai/venv/lib/python3.10/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/luise/Documents/DataScience/Projects/UrbanEagle/urbaneagle_visio

--------------------------------------
## Transfer learn / training